In [1]:
import torch
import numpy as np
from pathlib import Path

from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import pairwise
import pandas as pd
from openai import OpenAI
import openai

from sklearn.metrics import accuracy_score, classification_report, balanced_accuracy_score
from sklearn.metrics import balanced_accuracy_score

import random

In [2]:
embeddings_data_path = Path("../data/embeddings/")
input_data_path = Path("../data/data_splits_stratified/6-2-2_all_classes_enriched_with_kw")

## Load Embeddings and Calculate Similarities

### Load 
-> each row represents the text from one sample embedded into a 768-demnsional vector

In [3]:
embeddings_train = np.load(
    embeddings_data_path / "embeddings_microsoft_BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext_train_ds.npy"
)
embeddings_test = np.load(
    embeddings_data_path / "embeddings_microsoft_BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext_test_ds.npy"
)

In [4]:
embeddings_train.shape

(1191, 768)

In [5]:
embeddings_test.shape

(404, 768)

### Compute similarity between test dataset elements to all train elements


In [6]:
dist_matrix = pairwise_distances(embeddings_test, embeddings_train, metric='sqeuclidean')
dist_matrix.shape

(404, 1191)

In [7]:
dist_matrix_cosine = pairwise.cosine_similarity(X=embeddings_test, Y=embeddings_train)
dist_matrix.shape

(404, 1191)

The matrix has the dimensions 404 (test data elements) x 1191 (train data elements). Each of the 1191 values per test row is the similarity score between the test element to the train elements.

In [8]:
dist_matrix_cosine

array([[0.98506499, 0.98830865, 0.98870777, ..., 0.98741564, 0.98696952,
        0.98295794],
       [0.99007619, 0.99129481, 0.9916276 , ..., 0.99085124, 0.99138424,
        0.98798618],
       [0.98900018, 0.99066293, 0.9915621 , ..., 0.98766824, 0.99073964,
        0.98834762],
       ...,
       [0.98882802, 0.9873871 , 0.98857115, ..., 0.9928928 , 0.98869445,
        0.98763254],
       [0.98709908, 0.987791  , 0.9864768 , ..., 0.9930578 , 0.98749872,
        0.98896695],
       [0.98904398, 0.98862152, 0.98700841, ..., 0.9863798 , 0.99105832,
        0.99154065]])

### Find closest neighbours from the train dataset to each test example

In [9]:
values, indices = torch.topk(-torch.from_numpy(dist_matrix_cosine), k=3, dim=-1)

In [10]:
indices.shape

torch.Size([404, 3])

The indices have the dimension 404 (test data elements) x 3 (top k=3 closest train data elements).

In [11]:
indices

tensor([[ 128, 1032,  413],
        [ 128, 1032,  413],
        [ 128,  413, 1032],
        ...,
        [ 128,   11, 1032],
        [  11,  128,  356],
        [ 128,  356, 1032]])

In [12]:
df_train = pd.read_csv(input_data_path/ 'train.csv')
df_test = pd.read_csv(input_data_path/ 'test.csv')


In [13]:
df_test[df_test['accepted_label'] == 'In-vitro-study']

,idx,pmid,journal_name,title,abstract,keywords,accepted_label,multi_label,binary_label
265,266,24036210,Cellular signalling,Homer1 knockdown protects dopamine neurons thr...,Homer1 protein is an important scaffold protei...,Calcium channel| Endoplasmic reticulum| Homer|...,In-vitro-study,8,0
266,267,31852295,"Artificial cells, nanomedicine, and biotechnology",Fyn gene silencing reduces oligodendrocytes ap...,This study aimed to investigate the effect of ...,ERK1/2| Epilepsy| Fyn| apoptosis| gene silenci...,In-vitro-study,8,0
267,268,14655759,"Brain pathology (Zurich, Switzerland)",TRAIL triggers apoptosis in human malignant gl...,Many malignant glioma cells express death rece...,NaN,In-vitro-study,8,0
268,269,1720637,No to shinkei = Brain and nerve,[Immunohistochemical study of glial cytoplasmi...,"Recently, glial cytoplasmic inclusion (GCI) ha...",NaN,In-vitro-study,8,0
269,270,34477946,Journal of cancer research and clinical oncology,Selective induction of rapid cytotoxic effect ...,PURPOSE: The mechanisms underlying anticancer ...,Brain tumor| Cell culture| Noninvasive treatme...,In-vitro-study,8,0
270,271,28923314,Journal of virological methods,A simple method for isolation of cell-associat...,A common method for cell-associated virus isol...,Cell-associated virus| Cell-free extract| Infe...,In-vitro-study,8,0
271,272,27030558,"Network (Bristol, England)",Molecular interaction studies of green tea cat...,Green tea catechins have extensively been stud...,Catechins| molecular docking| pharmacokinetic ...,In-vitro-study,8,0
272,273,26568330,"Experimental biology and medicine (Maywood, N.J.)",Penetration and intracellular uptake of poly(g...,Nanoparticle (NP) drug delivery systems may po...,Bionanoscience| brain| polymer nanoparticles| ...,In-vitro-study,8,0
273,274,23926278,"Journal of neurology, neurosurgery, and psychi...",Sera from patients with multifocal motor neuro...,"In multifocal motor neuropathy (MMN), the dest...",BLOOD-BRAIN BARRIER| IMMUNOLOGY| NEUROCHEMISTR...,In-vitro-study,8,0
274,275,22975132,Brain research,Expression of CD200 in alternative activation ...,CD200 is a glycoprotein that is expressed on t...,NaN,In-vitro-study,8,0


In [14]:
df_test.iloc[246]

idx                                                             247
pmid                                                       37905374
journal_name                                      Cognitive science
title             Repeat After Me? Both Children With and Withou...
abstract          Linguistic repetitions in children are concept...
keywords          Autism| Echolalia| Individual differences| Lan...
accepted_label                                            Remaining
multi_label                                                       0
binary_label                                                      0
Name: 246, dtype: object

In [15]:
indices[246]

tensor([ 128,  356, 1032])

In [16]:
df_train.iloc[1032]

idx                                                            1033
pmid                                                       21356158
journal_name                         International psychogeriatrics
title             Animal-assisted activity and emotional status ...
abstract          BACKGROUND: Preliminary studies suggest benefi...
keywords                                                        NaN
accepted_label                  Human-non-RCT-non-drug-intervention
multi_label                                                       2
binary_label                                                      0
Name: 1032, dtype: object

In [17]:
df_test.groupby('accepted_label').size()

accepted_label
Animal-drug-intervention                30
Animal-non-drug-intervention            16
Animal-other                            31
Clinical-study-protocol                 12
Human-RCT-drug-intervention             15
Human-RCT-non-drug-intervention         18
Human-RCT-non-intervention              11
Human-case-report                       33
Human-non-RCT-drug-intervention         36
Human-non-RCT-non-drug-intervention     42
Human-systematic-review                 22
In-vitro-study                          21
Non-systematic-review                   75
Remaining                              172
dtype: int64

## Init OpenAI API

In [18]:
def load_pass(file_path, key_to_find):
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split("=")
            if len(parts) == 2 and parts[0] == key_to_find:
                found_password = parts[1]
                break
    if found_password:
        print("Found password.")
        return found_password
    else:
        print("Password not found for key:", key_to_find)

In [19]:
openai.api_key = load_pass("./credentials.txt", "OPENAI")
client = OpenAI(api_key=openai.api_key)

Found password.


## Create Prompts

In [20]:
df_train['input_journal_title_abstract'] = '<journal>' + df_train['journal_name'] + '</journal>' + \
                                         '<title>' + df_train['title'] + '</title>' + \
                                         '<abstract>' + df_train['abstract'] + '</abstract>'
df_test['input_journal_title_abstract'] = '<journal>' + df_test['journal_name'] + '</journal>' + \
                                         '<title>' + df_test['title'] + '</title>' + \
                                         '<abstract>' + df_test['abstract'] + '</abstract>'

In [21]:
df_test

,idx,pmid,journal_name,title,abstract,keywords,accepted_label,multi_label,binary_label,input_journal_title_abstract
0,1,12047012,Schizophrenia bulletin,Can clinical practice guide a research agenda?,Articles from this issue of the Bulletin indic...,NaN,Non-systematic-review,1,0,<journal>Schizophrenia bulletin</journal><titl...
1,2,28832188,Future medicinal chemistry,Dual/multitargeted xanthone derivatives for Al...,"To date, the current therapy for Alzheimer's d...",Alzheimer's disease| Aβ aggregation| anticholi...,Non-systematic-review,1,0,<journal>Future medicinal chemistry</journal><...
2,3,17678496,Expert review of neurotherapeutics,Benefits of occupational therapy in stroke reh...,Stroke is the largest single cause of severe p...,NaN,Non-systematic-review,1,0,<journal>Expert review of neurotherapeutics</j...
3,4,25649308,Annals of the New York Academy of Sciences,The promise of ketamine for treatment-resistan...,Major depressive disorder (MDD) is one of the ...,antidepressant| bipolar disorder| ketamine| ma...,Non-systematic-review,1,0,<journal>Annals of the New York Academy of Sci...
4,5,6312596,La semaine des hopitaux : organe fonde par l'A...,[Clinical and pathogenic aspects of secondary ...,The secondary hyperlipoproteinemias are freque...,NaN,Non-systematic-review,1,0,<journal>La semaine des hopitaux : organe fond...
...,...,...,...,...,...,...,...,...,...,...
529,530,24507711,Journal of clinical neuroscience : official jo...,Validation of the Chinese version of the NUCOG...,We aimed to develop and validate a Chinese ver...,Cognition| Dementia| Epilepsy| Mini-Mental Sta...,Human-RCT-non-intervention,13,0,<journal>Journal of clinical neuroscience : of...
530,531,28063795,The Lancet. Infectious diseases,Effect of new tuberculosis diagnostic technolo...,BACKGROUND: Inadequate case detection results ...,NaN,Human-RCT-non-intervention,13,0,<journal>The Lancet. Infectious diseases</jour...
531,532,35488938,European child & adolescent psychiatry,The impact of school-based screening on servic...,Early detection and intervention can counterac...,Adolescents| Mental health problems| Risk beha...,Human-RCT-non-intervention,13,0,<journal>European child & adolescent psychiatr...
532,533,16158009,Dementia and geriatric cognitive disorders,Validating the DemTect with 18-fluoro-2-deoxy-...,OBJECTIVES: The first study to validate the di...,NaN,Human-RCT-non-intervention,13,0,<journal>Dementia and geriatric cognitive diso...


In [22]:
def create_prompt(df_train, df_test, test_index, example_indices):
    # Start the prompt with a task description (optional)
    prompt = "Classify this text, choosing one of these labels: Clinical-study-protocol, Human-systematic-review, Non-systematic-review, Human-RCT-non-drug-intervention, Human-RCT-drug-intervention, Human-RCT-non-intervention, Human-case-report, Human-non-RCT-non-drug-intervention, Human-non-RCT-drug-intervention, Animal-systematic-review, Animal-drug-intervention, Animal-non-drug-intervention, Animal-other, In-vitro-study, Remaining. Respond in json format with the key: gpt_label.\n\n"
    
    # Add examples from df_train
    for idx in example_indices:
        example_text = df_train.loc[idx, 'input_journal_title_abstract']
        example_label = df_train.loc[idx, 'accepted_label']
        prompt += f"Text: \"{example_text}\"\nCategory: {example_label}\n\n"
    
    # Add the test text needing classification
    test_text = df_test.loc[test_index, 'input_journal_title_abstract']
    prompt += f"Text: \"{test_text}\"\nCategory: "
    
    return prompt

In [23]:
example_i = 0
example_prompt = create_prompt(df_train, df_test, example_i, indices[example_i].tolist())
#example_prompt

In [24]:
len(example_prompt.split())

599

In [25]:
import time
from tqdm.auto import tqdm

DEFAULT_TEMPERATURE = 0

def create_prompt(df_train, example_indices, input_raw_text):
    prompt = "Classify this text, choosing one of these labels: Clinical-study-protocol, Human-systematic-review, Non-systematic-review, Human-RCT-non-drug-intervention, Human-RCT-drug-intervention, Human-RCT-non-intervention, Human-case-report, Human-non-RCT-non-drug-intervention, Human-non-RCT-drug-intervention, Animal-systematic-review, Animal-drug-intervention, Animal-non-drug-intervention, Animal-other, In-vitro-study, Remaining. Respond in json format with the key: gpt_label.\n\n"
    for idx in example_indices:
        example_text = df_train.loc[idx, 'input_journal_title_abstract']
        example_label = df_train.loc[idx, 'accepted_label']
        prompt += f"Text: \"{example_text}\"\nCategory: {example_label}\n\n"
    prompt += f"Text: \"{input_raw_text}\"\nCategory: "
    return prompt

def query_gpt(df_train, input_raw_text, example_indices, gpt_model="gpt-3.5-turbo", temperature=DEFAULT_TEMPERATURE, max_retries=5, retry_delay=3):
    prompt_text = create_prompt(df_train, example_indices, input_raw_text)
    system_msg = f"You are an expert assistant specialized in text classification of PubMed abstracts."

    retries = 0
    while retries < max_retries:
        print("Trying to call OpenAI API...")
        try:
            completion = client.chat.completions.create(
                model=gpt_model,  
                response_format={"type": "json_object"},
                temperature=temperature,
                messages=[
                    {"role": "system", "content": system_msg},
                    {"role": "user", "content": prompt_text}
                ]
            )
            return completion.choices[0].message.content
        except Exception as e:
            print(f"OpenAI API returned an error: {e}")
            time.sleep(retry_delay)
            retries += 1

    raise RuntimeError("Max retries reached. Unable to complete the API call.")

def apply_gpt_with_progress(df_train, test_data_series, example_indices_tensor=None, num_samples=3, use_random=False, model="gpt-3.5-turbo"):
    results = []
    total_items = len(test_data_series)
    with tqdm(total=total_items, desc="Processing dataset") as pbar:
        for i, text in enumerate(test_data_series):
            if use_random:
                example_indices = random.sample(range(len(df_train)), num_samples)
            else:
                example_indices = example_indices_tensor[i].tolist()
            print("Retrieved in-context learning examples with idx: ", example_indices)
            result = query_gpt(df_train, text, example_indices, model)
            results.append(result)
            pbar.update(1)
    return results


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
# Example usage:
df_test[f'gpt_predictions_in_context'] = apply_gpt_with_progress(df_train, df_test['input_journal_title_abstract'], indices)

Processing dataset:   0%|                               | 0/534 [00:00<?, ?it/s]

Retrieved in-context learning examples with idx:  [128, 1032, 413]
Trying to call OpenAI API...


Processing dataset:   0%|                       | 1/534 [00:02<21:40,  2.44s/it]

Retrieved in-context learning examples with idx:  [128, 1032, 413]
Trying to call OpenAI API...


Processing dataset:   0%|                       | 2/534 [00:03<12:48,  1.44s/it]

...

Retrieved in-context learning examples with idx:  [11, 128, 356]
Trying to call OpenAI API...


Processing dataset:  75%|███████████████▊     | 403/534 [05:29<01:45,  1.25it/s]

Retrieved in-context learning examples with idx:  [128, 356, 1032]
Trying to call OpenAI API...


Processing dataset:  76%|███████████████▉     | 404/534 [05:30<01:46,  1.22it/s]


IndexError: index 404 is out of bounds for dimension 0 with size 404

In [ ]:
df_test.to_csv(f"predictions/{model}_enriched_kw_test_outputs_in_context_ICL.csv") #save k-nearest-neihgbor_ICL to csv

df_test[f'gpt_predictions_in_context_random'] = apply_gpt_with_progress(df_train, df_test['input_journal_title_abstract'], use_random=True)

In [ ]:
df_test.to_csv(f"predictions/{model}_enriched_kw_test_outputs_random_ICL.csv") #save random_ICL to csv

## Evaluate

In [ ]:
df_test_to_eval = df_test.copy()
df_test_to_eval.head()

In [ ]:
prompt_ids_to_test = ["in_context", "in_context_random"]

In [ ]:
labels = ["Human-systematic-review", "Human-RCT-drug-intervention", "Human-RCT-non-drug-intervention", "Human-RCT-non-intervention", "Human-case-report", "Human-non-RCT-drug-intervention", "Human-non-RCT-non-drug-intervention", "Animal-systematic-review", "Animal-drug-intervention", "Animal-non-drug-intervention", "Animal-other", "Non-systematic-review", "In-vitro-study", "Clinical-study-protocol", "Remaining"]

label_to_numerical = {label: i for i, label in enumerate(labels)}
label_to_numerical["label missing"] = -1

In [ ]:
Map predictions to numerical

In [ ]:
label_to_numerical = {
    'Remaining': 0,
    'Non-systematic-review': 1,
    'Human-non-RCT-non-drug-intervention': 2,
    'Human-non-RCT-drug-intervention': 3,
    'Human-case-report': 4,
    'Animal-other': 5,
    'Animal-drug-intervention': 6,
    'Human-systematic-review': 7,
    'In-vitro-study': 8,
    'Human-RCT-non-drug-intervention': 9,
    'Animal-non-drug-intervention': 10,
    'Human-RCT-drug-intervention': 11,
    'Clinical-study-protocol': 12,
    'Human-RCT-non-intervention': 13
}

In [ ]:
numerical_to_label = {v: f"{v} - {k}" for k, v in label_to_numerical.items()}

In [ ]:
import difflib

def map_label_fuzzy(label, label_dict):
    #label = label.lower().replace('-', '').replace('_', ' ').strip()
    best_match = difflib.get_close_matches(label, label_dict.keys(), n=1, cutoff=0.6)
    if best_match:
        #print(f'{label} matched to {best_match[0]}')
        return label_dict[best_match[0]]
    return -1

In [ ]:
map_label_fuzzy("RCT-drug-intervention", label_to_numerical)

In [ ]:
def map_label_to_numerical(label, label_dict):
    # Check if label is a dictionary
    if isinstance(label, dict):
        # Extract the label with the highest score/probability
        highest_label = max(normalized_label, key=label.get)
        return label_dict.get(highest_label, -1)
    else:
         # Normalize label
        normalized_label = label.replace(',', '').strip().replace(' ', '-')#.strip()
        # Directly map string labels to numerical IDs
        numerical_label = label_dict.get(label, -1)
        # Fuzzy match if no direct mapping possible
        if numerical_label == -1:
            numerical_label = map_label_fuzzy(normalized_label, label_dict)
        # If fuzzy match did not work, check if the label string contains the key 'label' and use it to split the string; keep everything to the right as the potential label
        if numerical_label == -1:
            if 'label' in label:
                label_part = label.split('label')[1]
                numerical_label = map_label_fuzzy(label_part, label_dict)
        return numerical_label
        
# Convert accepted labels to numerical
df['accepted_label_numerical'] = df['accepted_label'].apply(lambda x: map_label_to_numerical(x, label_to_numerical))


for col in prediction_columns:
    df[f'{col}_numerical'] = df[col].apply(lambda x: map_label_to_numerical(x, label_to_numerical))

df.head()

In [ ]:
Important: some labels from GPT could not be mapped to a target label

In [ ]:
rows_with_minus_one = df[(df == -1).any(axis=1)]
rows_with_minus_one

In [ ]:
def evaluate_predictions(df, target_label_col, prompt_ids_to_eval, model, eval_type, label_to_numerical, numerical_to_label): ## Initialize a list to hold DataFrame for each report and summary statistics
    report_dfs = []
    summary_stats = []

    # Iterate over each GPT prediction column
    for prompt_id in prompt_ids_to_eval:
        print("Evaluating ", prompt_id)
        prediction_col = f'gpt_predictions_{prompt_id}_clean'

    df_test_to_eval[prediction_col] = df_test_to_eval[f'gpt_predictions_{prompt_id}'].apply(
            lambda x: json.loads(x)['gpt_label'] if isinstance(x, str) and 'gpt_label' in json.loads(x) else x)
    
        # Map GPT predictions to numerical values
        df_test_to_eval[f'{prediction_col}_numerical'] = df_test_to_eval[prediction_col].apply(map_label_to_numerical)

        # Extract arrays for evaluation
        y_true = df_test_to_eval['accepted_label_numerical'].values
        y_pred = df_test_to_eval[f'{prediction_col}_numerical'].values
    
        # Calculate metrics
        accuracy = accuracy_score(y_true, y_pred)
        accuracy_balanced = balanced_accuracy_score(y_true, y_pred)
        report = classification_report(y_true, y_pred, output_dict=True, zero_division=0, labels=range(len(labels)), target_names=labels)
    
        # Create DataFrame from report
        report_df = pd.DataFrame(report).transpose()
        report_df['Prompt ID'] = prompt_id  # Add column to indicate the prompt ID
        report_dfs.append(report_df)
    
        # Extract summary statistics (average precision, recall, F1)
        summary = report_df.loc['weighted avg', ['precision', 'recall', 'f1-score']].to_dict()
        summary['Prompt ID'] = prompt_id
        summary_stats.append(summary)

        # Plotting confusion matrix
        plt.figure(figsize=(10, 6))
        ax = sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(len(label_to_numerical)), yticklabels=range(len(label_to_numerical)))
        ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=13)
        ax.set_xticklabels(ax.get_xticklabels(), rotation=0, fontsize=13)
        plt.title(f'Confusion Matrix for Model {model} and ICL-Prompt {prompt_id}', fontsize=14)
        plt.xlabel('Predicted Labels', fontsize=13)
        plt.ylabel('True Labels', fontsize=13)

        # Add an inset with label mapping
        textstr = '\n'.join([f'{v}: {k}' for k, v in label_to_numerical.items()])
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        ax.text(1.16, 1.0, textstr, transform=ax.transAxes, fontsize=10, verticalalignment='top', bbox=props)
        plt.tight_layout()
        plt.savefig(f'plots/confusion_matrix_{model}_enriched_kw_test_{prompt_id}_{eval_type}_ICL.pdf')  # Save to PDF

        # Combine all report DataFrames
        all_reports_df = pd.concat(report_dfs)

        # Create a summary table for average precision, recall, and F1-score
        summary_df = pd.DataFrame(summary_stats)

        # Save results to CSV files
        all_reports_df.to_csv(f"evaluations/{model}_enriched_kw_test_per_class_{'_'.join(prompt_ids_to_eval)}_{eval_type}_ICL.csv")
        summary_df.to_csv(f"evaluations/{model}_enriched_kw_test_summary_{'_'.join(prompt_ids_to_eval)}_{eval_type}_ICL.csv")

In [ ]:
all_reports_df

In [ ]:
summary_df

In [ ]:
print(summary_df)